# build_data

we build our data set here.

In [1]:
#FIXME: in original data, we have pid starts from 0, but here we need to set pid starts from 1

In [1]:
#1. vocabulary construction.
input_path='../../PerGen/code/OpenNMT-py/src_data_full_feat_tf_resplited_review'

# 1.1 read src data.
def read_src_data(file,vocab_dict):
    src=[]
    with open(file,'r') as fin:
        lines = fin.readlines()
        for line in lines:
            line=line.strip().split(' ')
            src.append(line)
            for word in line:
                if word not in vocab_dict:
                    vocab_dict[word]=0
                vocab_dict[word]+=1
    return src,vocab_dict

# 1.2 read tgt data.
def read_tgt_data(file,vocab_dict):
    tgt=[]
    with open(file,'r') as fin:
        lines = fin.readlines()
        for i,line in enumerate(lines):
            line=line.strip().split(' ')
            pid=None
            tgt_line=[]
            for word_pid in line:
#                 print(word_pid)
                word,pid=word_pid.split('￨')

                tgt_line.append(word)
                if word not in vocab_dict:
                    vocab_dict[word]=0
                vocab_dict[word]+=1
            tgt.append({'line':tgt_line,'pid':int(pid)+1})
            
            #FIXME: in original data, we have pid starts from 0, but here we need to set pid starts from 1
            
    return tgt,vocab_dict
def build_vocab(vocab_dict,min_frequency):
    ## we need to filter out those unfrequent words out.
    vocab_index_dict={}
    index_vocab_dict={}
    # 1. sort the vocab
    sorted_vocab=sorted(vocab_dict.items(), key=lambda item:item[1],reverse=True)
    
    # 2. build the index 
    UNKnown_index=1
    index_vocab_dict[UNKnown_index]='UNKnown'
    index=2
    for word,frequency in sorted_vocab:
        if frequency>min_frequency:
            vocab_index_dict[word]=index
            index_vocab_dict[index]=word
            index+=1
        else:
            vocab_index_dict[word]=UNKnown_index
    print("words num in vocab:",index-1)
    return vocab_index_dict,index_vocab_dict
        
def encode_sentence(sent,vocab_index_dict):
    codes=[]
    for word in sent:
        index=vocab_index_dict[word]
        codes.append(index)
    assert len(codes)==len(sent)
    return codes

def output_one_set(src,tgt,file):
    assert len(src)==len(tgt)
    with open(file,'w') as fout:
        for i in range(len(src)):
            src_sent=src[i]
            pair=tgt[i]
            tgt_sent=pair['line']
            tgt_pid=pair['pid']
            src_sent=encode_sentence(src_sent,vocab_index_dict)
            tgt_sent=encode_sentence(tgt_sent,vocab_index_dict)
            fout.write(str(tgt_pid)+' ')
            src_str=''
            for index in src_sent:
                src_str+=str(index)
                src_str+=' '
            src_str=src_str[:-1]
            tgt_str=''
            for index in tgt_sent:
                tgt_str+=str(index)
                tgt_str+=' '
            tgt_str=tgt_str[:-1]
            fout.write(src_str)
            fout.write('|')
            fout.write(str(tgt_pid)+' ')
            fout.write(tgt_str+'\n')
    print('finish',file,len(src),'')
            
def output_vocab_file(index_vocab_dict,file):
    with open(file,'w') as fout:
        for i in range(1,len(index_vocab_dict)+1):
            word=index_vocab_dict[i]
            fout.write(word+'\n')    
    print(len(index_vocab_dict),'words saved in',file)            
    
# 4. also encode the file
def back_encode_sentence(inds,index_vocab_dict):
    sent=''
    for ind in inds:
        sent+=index_vocab_dict[ind]
        sent+=' '
    return sent[:-1]
def encode_output(src,tgt,index_vocab_dict,file):
    with open(file,'w') as fout:
        assert len(src)==len(tgt)
        for i in range(len(src)):
            src_sent=src[i]
            pair=tgt[i]
            tgt_sent=pair['line']
            tgt_pid=pair['pid']
            src_sent=encode_sentence(src_sent,vocab_index_dict)
            tgt_sent=encode_sentence(tgt_sent,vocab_index_dict)
            src_sent=back_encode_sentence(src_sent,index_vocab_dict)
            tgt_sent=back_encode_sentence(tgt_sent,index_vocab_dict)
            output=str(tgt_pid)+' '+src_sent+'￨'+str(tgt_pid)+' '+tgt_sent+'\n'
            fout.write(output)
    print('finish',file,'with',len(src),'lines')
            
    

In [2]:
# read files

vocab_dict={}
file=input_path+'/src-test.txt'
src_test,vocab_dict=read_src_data(file,vocab_dict)
file=input_path+'/src-train.txt'
src_train,vocab_dict=read_src_data(file,vocab_dict)
file=input_path+'/src-val.txt'
src_val,vocab_dict=read_src_data(file,vocab_dict)



file=input_path+'/src-test-full.txt'
src_test_full,vocab_dict=read_src_data(file,vocab_dict)
file=input_path+'/src-val-full.txt'
src_val_full,vocab_dict=read_src_data(file,vocab_dict)


file=input_path+'/tgt-test.txt'
tgt_test,vocab_dict=read_tgt_data(file,vocab_dict)
file=input_path+'/tgt-test-full.txt'
tgt_test_full,vocab_dict=read_tgt_data(file,vocab_dict)

file=input_path+'/tgt-val.txt'
tgt_val,vocab_dict=read_tgt_data(file,vocab_dict)
file=input_path+'/tgt-val-full.txt'
tgt_val_full,vocab_dict=read_tgt_data(file,vocab_dict)

file=input_path+'/tgt-train.txt'
tgt_train,vocab_dict=read_tgt_data(file,vocab_dict)

In [14]:
# 2. build vocab

vocab_index_dict,index_vocab_dict=build_vocab(vocab_dict,min_frequency=6)

words num in vocab: 42182


In [21]:
# 3. output file

output_one_set(src_test,tgt_test,file='../data/src_data_full_feat_tf_resplited_review/test.txt')

output_one_set(src_train,tgt_train,file='../data/src_data_full_feat_tf_resplited_review/train.txt')

output_one_set(src_val,tgt_val,file='../data/src_data_full_feat_tf_resplited_review/valid.txt')

output_vocab_file(index_vocab_dict,file='../data/src_data_full_feat_tf_resplited_review/vocabulary',)


finish ../data/src_data_full_feat_tf_resplited_review/test.txt 4167 
finish ../data/src_data_full_feat_tf_resplited_review/train.txt 960114 
finish ../data/src_data_full_feat_tf_resplited_review/valid.txt 4167 
42182 words saved in ../data/src_data_full_feat_tf_resplited_review/vocabulary


In [12]:
# 4. also encode the file
encode_output(src_test,tgt_test,index_vocab_dict,file='../data/src_data_full_feat_tf_resplited_review/test_ref.txt')
encode_output(src_val,tgt_val,index_vocab_dict,file='../data/src_data_full_feat_tf_resplited_review/val_ref.txt')
encode_output(src_train,tgt_train,index_vocab_dict,file='../data/src_data_full_feat_tf_resplited_review/train_ref.txt')


finish ../data/src_data_full_feat_tf_resplited_review/test_ref.txt with 4167 lines
finish ../data/src_data_full_feat_tf_resplited_review/val_ref.txt with 4167 lines
finish ../data/src_data_full_feat_tf_resplited_review/train_ref.txt with 960114 lines


In [16]:
# 2. build vocab

vocab_index_dict,index_vocab_dict=build_vocab(vocab_dict,min_frequency=4)

words num in vocab: 49844


In [17]:
# 3. output file

output_one_set(src_test,tgt_test,file='../data/src_data_full_feat_tf_resplited_review_50k/test.txt')

output_one_set(src_test_full,tgt_test_full,file='../data/src_data_full_feat_tf_resplited_review_50k/test_full.txt')

output_one_set(src_val,tgt_val,file='../data/src_data_full_feat_tf_resplited_review_50k/valid.txt')

output_one_set(src_val_full,tgt_val_full,file='../data/src_data_full_feat_tf_resplited_review_50k/valid_full.txt')

output_one_set(src_train,tgt_train,file='../data/src_data_full_feat_tf_resplited_review_50k/train.txt')



output_vocab_file(index_vocab_dict,file='../data/src_data_full_feat_tf_resplited_review_50k/vocabulary',)

# 4. also encode the file
encode_output(src_test,tgt_test,index_vocab_dict,file='../data/src_data_full_feat_tf_resplited_review_50k/test_ref.txt')
encode_output(src_val,tgt_val,index_vocab_dict,file='../data/src_data_full_feat_tf_resplited_review_50k/val_ref.txt')
encode_output(src_test_full,tgt_test_full,index_vocab_dict,file='../data/src_data_full_feat_tf_resplited_review_50k/test_full_ref.txt')
encode_output(src_val_full,tgt_val_full,index_vocab_dict,file='../data/src_data_full_feat_tf_resplited_review_50k/val_full_ref.txt')

encode_output(src_train,tgt_train,index_vocab_dict,file='../data/src_data_full_feat_tf_resplited_review_50k/train_ref.txt')



finish ../data/src_data_full_feat_tf_resplited_review_50k/test.txt 4167 
finish ../data/src_data_full_feat_tf_resplited_review_50k/test_full.txt 16668 
finish ../data/src_data_full_feat_tf_resplited_review_50k/valid.txt 4167 
finish ../data/src_data_full_feat_tf_resplited_review_50k/valid_full.txt 16668 
finish ../data/src_data_full_feat_tf_resplited_review_50k/train.txt 960114 
49844 words saved in ../data/src_data_full_feat_tf_resplited_review_50k/vocabulary
finish ../data/src_data_full_feat_tf_resplited_review_50k/test_ref.txt with 4167 lines
finish ../data/src_data_full_feat_tf_resplited_review_50k/val_ref.txt with 4167 lines
finish ../data/src_data_full_feat_tf_resplited_review_50k/test_full_ref.txt with 16668 lines
finish ../data/src_data_full_feat_tf_resplited_review_50k/val_full_ref.txt with 16668 lines
finish ../data/src_data_full_feat_tf_resplited_review_50k/train_ref.txt with 960114 lines


In [ ]:
# python train.py \
# --data_folder 'data/src_data_full_feat_tf_resplited_review' \
# --SpeakerMode --batch_size 1 --eval_steps 1 --layers 1 --no_save


# python train.py \
# --data_folder 'data/src_data_full_feat_tf_resplited_review' \
# --SpeakerMode --batch_size 1 --eval_steps 1 --layers 1 --dev_file 'valid_testing.txt'  --no_save


# python train.py \
# --data_folder 'data/src_data_full_feat_tf_resplited_review' \
# --SpeakerMode --batch_size 16 --max_iter 300000 --layers 3 --no_save

CUDA_VISIBLE_DEVICES=0 python train.py \
--data_folder 'data/src_data_full_feat_tf_resplited_review' --SpeakerMode \
--batch_size 200 --layers 2 --epochs 10 --eval_steps 700 --save_steps 2 --train_size 960114

CUDA_VISIBLE_DEVICES=1 python train.py \
--data_folder 'data/src_data_full_feat_tf_resplited_review' \
--batch_size 200 --layers 2 --epochs 10 --eval_steps 700 --save_steps 2 --train_size 960114

CUDA_VISIBLE_DEVICES=0 python train.py \
--data_folder 'data/src_data_full_feat_tf_resplited_review' --SpeakerMode \
--batch_size 200 --layers 2 --epochs 10 --eval_steps 700 --save_steps 2 --train_size 960114



CUDA_VISIBLE_DEVICES=0 python train.py \
--data_folder 'data/src_data_full_feat_tf_resplited_review_50k' \
--save_folder 'save/s_rand' \
--save_prefix 'dim_40_lstm_512_layer_2.model.' \
--output_file 's_rand_dim_40_lstm_512_layer_2.log' \
--SpeakerMode \
--batch_size 200 --layers 2 --epochs 10 --eval_steps 700 --save_steps 2 --train_size 960114 \
--PersonaDim 40 --debug


CUDA_VISIBLE_DEVICES=0 python decode.py \
--data_folder 'data/src_data_full_feat_tf_resplited_review' \
--model_folder 'save/testing' --model_name 'model10' --params_name 'params' \
--output_folder 'outputs' --SpeakerMode --output_file 'SpeakerID1A1'

In [ ]:
CUDA_VISIBLE_DEVICES=0 python train.py \
--data_folder 'data/src_data_full_feat_tf_resplited_review_50k' \
--save_folder 'save/s_rand' \
--save_prefix 'dim_30_lstm_512_layer_2.model.' \
--output_file 's_rand_dim_30_lstm_512_layer_2.log' \
--SpeakerMode \
--batch_size 200 --layers 2 --epochs 10 --eval_steps 700 --save_steps 2 --train_size 960114 \
--PersonaDim 30 

CUDA_VISIBLE_DEVICES=0 python decode.py --decode_file 'test.txt' \
--data_folder 'data/src_data_full_feat_tf_resplited_review_50k' \
--model_folder 'save/s_rand' --model_name 'dim_30_lstm_512_layer_2.model.best' --params_name 'params' \
--output_folder 'outputs/s_rand' --SpeakerMode --output_file 's_rand_dim_30_lstm_512_layer_2_debug.txt'
CUDA_VISIBLE_DEVICES=0 python decode.py --decode_file 'test_full.txt' \
--data_folder 'data/src_data_full_feat_tf_resplited_review_50k' \
--model_folder 'save/s_rand' --model_name 'dim_30_lstm_512_layer_2.model.best' --params_name 'params' \
--output_folder 'outputs/s_rand' --SpeakerMode --output_file 's_rand_dim_30_lstm_512_layer_2_debug.txt'


CUDA_VISIBLE_DEVICES=0 python decode.py --decode_file 'valid.txt' \
--data_folder 'data/src_data_full_feat_tf_resplited_review_50k' \
--model_folder 'save/s_rand' --model_name 'dim_30_lstm_512_layer_2.model.best' --params_name 'params' \
--output_folder 'outputs/s_rand' --SpeakerMode --output_file 's_rand_dim_30_lstm_512_layer_2_debug.txt'
CUDA_VISIBLE_DEVICES=0 python decode.py --decode_file 'valid_full.txt' \
--data_folder 'data/src_data_full_feat_tf_resplited_review_50k' \
--model_folder 'save/s_rand' --model_name 'dim_30_lstm_512_layer_2.model.best' --params_name 'params' \
--output_folder 'outputs/s_rand' --SpeakerMode --output_file 's_rand_dim_30_lstm_512_layer_2_debug.txt'


In [ ]:
CUDA_VISIBLE_DEVICES=1 python train.py \
--data_folder 'data/src_data_full_feat_tf_resplited_review_50k' \
--save_folder 'save/s_tf' \
--save_prefix 'dim_30_lstm_512_layer_2.model.' \
--output_file 's_tf_dim_30_lstm_512_layer_2.log' \
--SpeakerMode \
--PersonaEmbFiles '../PerGen/src_data_full_1M_PCA_dim_384/FACTOR2_iter_200_15x30x20_0.0005_1_4' \
--batch_size 200 --layers 2 --epochs 10 --eval_steps 700 --save_steps 2 --train_size 960114 \
--PersonaDim 30 



CUDA_VISIBLE_DEVICES=1 python decode.py --decode_file 'test.txt' \
--data_folder 'data/src_data_full_feat_tf_resplited_review_50k' \
--model_folder 'save/s_tf' --model_name 'dim_30_lstm_512_layer_2.model.best' --params_name 'params' \
--SpeakerMode  \
--output_folder 'outputs/s_tf' --output_file 's_tf_dim_30_lstm_512_layer_2.txt'
CUDA_VISIBLE_DEVICES=1 python decode.py --decode_file 'test_full.txt' \
--data_folder 'data/src_data_full_feat_tf_resplited_review_50k' \
--model_folder 'save/s_tf' --model_name 'dim_30_lstm_512_layer_2.model.best' --params_name 'params' \
--SpeakerMode  \
--output_folder 'outputs/s_tf' --output_file 's_tf_dim_30_lstm_512_layer_2.txt'

CUDA_VISIBLE_DEVICES=1 python decode.py --decode_file 'valid.txt' \
--data_folder 'data/src_data_full_feat_tf_resplited_review_50k' \
--model_folder 'save/s_tf' --model_name 'dim_30_lstm_512_layer_2.model.best' --params_name 'params' \
--SpeakerMode  \
--output_folder 'outputs/s_tf' --output_file 's_tf_dim_30_lstm_512_layer_2.txt'
CUDA_VISIBLE_DEVICES=1 python decode.py --decode_file 'valid_full.txt' \
--data_folder 'data/src_data_full_feat_tf_resplited_review_50k' \
--model_folder 'save/s_tf' --model_name 'dim_30_lstm_512_layer_2.model.best' --params_name 'params' \
--SpeakerMode  \
--output_folder 'outputs/s_tf' --output_file 's_tf_dim_30_lstm_512_layer_2.txt'



In [ ]:
python train.py \
--data_folder 'data/src_data_full_feat_tf_resplited_review_50k' \
--save_folder 'save/seq2seq' \
--save_prefix 'lstm_512_layer_2.model.' \
--output_file 'seq2seq_lstm_512_layer_2.log' \
--batch_size 400 --layers 2 --epochs 10 --eval_steps 350 --save_steps 2 --train_size 960114 \
--gpu '0,1,2,3'
